In [1]:
import torch
print(torch.__version__)

1.7.0+cu110


In [2]:
!pip install pandas

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
!pip3 install datasets
!pip3 install rouge_score
!pip3 install git+https://github.com/huggingface/transformers
!pip3 install sentencepiece

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ysyk6ri0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.7.0.dev0-py3-none-any.whl size=2325994 sha256=5a983a5dc4e5ae93c77e6e32fa7db3ffbb90c8fb77db7ea4d92d05f55cb6e192
  Stored in directory: /tmp/pip-ephem-wheel-cache-4a0qeg4q/wheels/5a/0a/d0/eb8d0ea1d7d02156f8675d6e5dfa52c03601cbe377290db8dc
Successfully built transformers
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
train=pd.read_csv('conditioned_data.csv')

In [4]:
train.shape


(10589, 12)

In [5]:
train=train[['abstract','extractedsam']]

In [6]:
train.rename(columns = {'extractedsam':'articles'}, inplace = True)

In [7]:
train.isnull().sum(), train.shape

(abstract    0
 articles    0
 dtype: int64,
 (10589, 2))

In [8]:
test=train[10389:10588]
val=train[10000:10389]
train=train[0:10000]

In [10]:
import re
def finalclean(c):
    c = re.sub(r'[^A-Za-z0-9\n., %]+', " ",c)
    c=re.sub(r'Table [0-9]*',"", c)
    return c

def removeurls(text):
    result = re.sub(r"http\S+", "", text)
    result = re.sub(r"www\S+", "", result)
    return result
def removbrackets(text):
    text=re.sub("[\(\[].*?[\)\]]", "", text)
    return text
def misc_clean(text):
    d=re.sub('[a-z0-9.-]*/[a-z0-9.-]*'," ", text)
    return d

def single(text):
   # Single character removal
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)
    return text

In [12]:
train['articles'] = train['articles'].apply(lambda x: removeurls(x))
val['articles'] = val['articles'].apply(lambda x: removeurls(x))
train['abstract'] = train['abstract'].apply(lambda x: removeurls(x))
val['abstract'] = val['abstract'].apply(lambda x: removeurls(x))
test['articles'] = test['articles'].apply(lambda x: removeurls(x))

train['articles'] = train['articles'].apply(lambda x: removbrackets(x))
val['articles'] = val['articles'].apply(lambda x: removbrackets(x))
train['abstract'] = train['abstract'].apply(lambda x: removbrackets(x))
val['abstract'] = val['abstract'].apply(lambda x: removbrackets(x))
test['articles'] = test['articles'].apply(lambda x: removbrackets(x))

train['articles'] = train['articles'].apply(lambda x: misc_clean(x))
val['articles'] = val['articles'].apply(lambda x: misc_clean(x))
train['abstract'] = train['abstract'].apply(lambda x: misc_clean(x))
val['abstract'] = val['abstract'].apply(lambda x: misc_clean(x))
test['articles'] = test['articles'].apply(lambda x: misc_clean(x))

train['abstract'] = train['abstract'].apply(lambda x: finalclean(x))
val['abstract'] = val['abstract'].apply(lambda x: finalclean(x))
train['articles'] = train['articles'].apply(lambda x: finalclean(x))
val['articles'] = val['articles'].apply(lambda x: finalclean(x))
test['articles'] = test['articles'].apply(lambda x: finalclean(x))

train['articles'] = train['articles'].apply(lambda x: single(x))
val['articles'] = val['articles'].apply(lambda x: single(x))
test['articles'] = test['articles'].apply(lambda x: single(x))

In [11]:
#delete later
test['abstract'] = test['abstract'].apply(lambda x: single(x))
test['abstract'] = test['abstract'].apply(lambda x: removeurls(x))
test['abstract'] = test['abstract'].apply(lambda x: removbrackets(x))
test['abstract'] = test['abstract'].apply(lambda x: misc_clean(x))
test['abstract'] = test['abstract'].apply(lambda x: finalclean(x))

In [12]:
#delete later
test['articles'] = test['articles'].apply(lambda x: single(x))
test['articles'] = test['articles'].apply(lambda x: removeurls(x))
test['articles'] = test['articles'].apply(lambda x: removbrackets(x))
test['articles'] = test['articles'].apply(lambda x: misc_clean(x))
test['articles'] = test['articles'].apply(lambda x: finalclean(x))

In [13]:
test.isnull().sum()

abstract    0
articles    0
dtype: int64

In [13]:
train.isnull().sum(), val.isnull().sum(), test.isnull().sum()

(abstract    0
 articles    0
 dtype: int64,
 abstract    0
 articles    0
 dtype: int64,
 abstract    0
 articles    0
 dtype: int64)

In [14]:
from datasets import Dataset
train = Dataset.from_pandas(train)
val = Dataset.from_pandas(val)
test = Dataset.from_pandas(test)

In [15]:
#delete later
from datasets import Dataset
test = Dataset.from_pandas(test)

In [16]:
#delete later
test

Dataset({
    features: ['abstract', 'articles'],
    num_rows: 199
})

In [15]:
train, val, test

(Dataset({
     features: ['abstract', 'articles'],
     num_rows: 10000
 }),
 Dataset({
     features: ['abstract', 'articles'],
     num_rows: 389
 }),
 Dataset({
     features: ['abstract', 'articles'],
     num_rows: 199
 }))

In [17]:
DATASET_NAME = "pubmed"
device = "cuda"

MODEL_ID = f"google/bigbird-pegasus-large-{DATASET_NAME}"
from transformers import AutoTokenizer, AutoModelForMaskedLM
  

from transformers import EncoderDecoderModel, BigBirdModel, BigBirdForCausalLM, BigBirdTokenizer
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
#from transformers import PegasusTokenizer, BigBirdPegasusModel
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer
import torch
model = BigBirdPegasusForConditionalGeneration.from_pretrained('summary/checkpoint-7500', gradient_checkpointing=True, use_cache=False).to(device)
tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-pubmed",use_fast=False)
#tokenizer = AutoTokenizer.from_pretrained("deepset/covid_bert_base")

In [20]:
def collate_fn(features):
    

    article = [x["articles"] for x in features]
  #section_names = [x["section_names"] for x in features]
    abstract = [x["abstract"] for x in features]

  # should not eliminate special tokens since question and context are should have `SEP` in middle
    inputs = tokenizer(article, return_tensors="pt", padding="max_length", truncation=True, max_length=4096)
    labels = tokenizer(abstract, return_tensors="pt", padding=True, truncation=True, max_length=256)

    return {
      "input_ids": inputs.input_ids,
      "attention_mask": inputs.attention_mask,
      #"decoder_input_ids": labels.input_ids,
      "labels": labels.input_ids,
      "decoder_attention_mask": labels.attention_mask,
  }

In [22]:
args = Seq2SeqTrainingArguments(
    output_dir="summary",
    overwrite_output_dir=False,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    # eval_steps=4000,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=5e-5,
    num_train_epochs=3,
    logging_strategy="epoch",
    logging_steps=4000,
    save_strategy="epoch",
    run_name="BBsummary",
    disable_tqdm=False,
    load_best_model_at_end=True,
    remove_unused_columns=False
    #fp16=True,
   
)

trainer = Seq2SeqTrainer(
      model=model,
      args=args,
      data_collator=collate_fn,
      train_dataset=train,
      eval_dataset=val)

In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.350900,2.688609
2,2.761300,2.542668
3,2.615300,2.509889


TrainOutput(global_step=7500, training_loss=2.9091727864583334, metrics={'train_runtime': 79407.7078, 'train_samples_per_second': 0.378, 'train_steps_per_second': 0.094, 'total_flos': 4.2533086298112e+17, 'epoch': 3.0})

In [1]:
trainer.save_model("bigbird_conditioned_extractive")

NameError: name 'trainer' is not defined

In [18]:
from datasets import load_dataset, load_metric
rouge = load_metric("rouge")

In [19]:
def generate_answer(batch):
    inputs_dict = tokenizer(batch["articles"], padding="max_length", max_length=4096, return_tensors="pt", truncation=True)
    inputs_dict = {k: inputs_dict[k].to(device) for k in inputs_dict}
    predicted_abstract_ids = model.generate(**inputs_dict, max_length=256, num_beams=5, length_penalty=0.8)
    batch["predicted_abstract"] = tokenizer.decode(predicted_abstract_ids[0], skip_special_tokens=True)
    print(batch["predicted_abstract"])
    return batch

In [20]:
dataset_small = test.select(range(100))
result_small = dataset_small.map(generate_answer)

rouge.compute(predictions=result_small["predicted_abstract"], references=result_small["abstract"])

Abstract The aim of the present study was to investigate the role of lymphocyte activation molecule as a receptor for peste des petits ruminants virus, a causative agent of peste des petits ruminants virus. In this study, small interfering I transfection was carried out using an adherent cell line, I95a, which was transfected with sialic acid conjugated anti human sialic acid conjugate. The effect of sialic acid conjugated anti human sialic acid was assessed using real time reverse transcriptase polymerase chain reaction. The results showed that sialic acid conjugated anti human sialic acid was effective in inhibiting the expression of lymphocyte activation molecule at the protein level and at the gene level. In addition, sialic acid conjugated anti human sialic acid was also effective in inhibiting the expression of virus titres at the protein level and at the gene level. We conclude that sialic acid conjugated anti human sialic acid was effective in inhibiting the expression of lymph

{'rouge1': AggregateScore(low=Score(precision=0.3802158934306959, recall=0.32420229084563706, fmeasure=0.3365428136501211), mid=Score(precision=0.41181609752198256, recall=0.3447891116807198, fmeasure=0.35708843465587115), high=Score(precision=0.4419624907500547, recall=0.3661939600493618, fmeasure=0.37737342177793304)),
 'rouge2': AggregateScore(low=Score(precision=0.11565439031380233, recall=0.09465918391462375, fmeasure=0.09934876583012872), mid=Score(precision=0.136266191081244, recall=0.10819993569764613, fmeasure=0.11486889303783614), high=Score(precision=0.16105961045290093, recall=0.12313157565779163, fmeasure=0.13262493397914935)),
 'rougeL': AggregateScore(low=Score(precision=0.23148177070367834, recall=0.19720361392211838, fmeasure=0.20429072731073655), mid=Score(precision=0.2525812036824868, recall=0.2126771375117904, fmeasure=0.21862751830110222), high=Score(precision=0.2778977072173677, recall=0.22811166496126853, fmeasure=0.23612616120222016)),
 'rougeLsum': AggregateSco

In [1]:
model_ckpt2 = create_model()

NameError: name 'create_model' is not defined

In [21]:
dataset_small = val.select(range(30))
result_small = dataset_small.map(generate_answer)

rouge.compute(predictions=result_small["predicted_abstract"], references=result_small["abstract"])

Abstract The porcine enteric coronavirus, porcine epidemic diarrhea virus, porcine deltacoronavirus and porcine deltacoronavirus are the causative agents of enteric and respiratory tract infections in piglets worldwide. In this review, we summarize the current knowledge on the pathogenesis of porcine enteric coronaviruses, focusing on the role of virus host interactions in their pathogenesis.
Abstract The development and characterization of a murine bovine hybrid which produces a bovine monoclonal antibody to the bovine enteric coronavirus lipopolysaccharide lipopolysaccharide is reported. The monoclonal antibody was prepared by affinity chromatography from pools of ascites fluid and was further purified by affinity chromatography and conjugated with horseradish peroxidase. The antibody was used in a diagnostic indirect immunofluorescence assay to detect antibodies to bovine spongiform encephalomyelitis.
Abstract The aim of the present study was to compare clinical signs, virological p

{'rouge1': AggregateScore(low=Score(precision=0.3756855904529516, recall=0.32437344248146416, fmeasure=0.33609048664333097), mid=Score(precision=0.42402972376446063, recall=0.3646903687143615, fmeasure=0.36236011805558643), high=Score(precision=0.4781014034468284, recall=0.40397713183511214, fmeasure=0.3886825238128921)),
 'rouge2': AggregateScore(low=Score(precision=0.10870285093408455, recall=0.10004761409904209, fmeasure=0.09990740146253824), mid=Score(precision=0.14302839025976435, recall=0.11926623193891636, fmeasure=0.12025148075088765), high=Score(precision=0.1947051584496048, recall=0.13983492543070958, fmeasure=0.1481351750014833)),
 'rougeL': AggregateScore(low=Score(precision=0.2284704687910609, recall=0.20403907497427456, fmeasure=0.20802140100841046), mid=Score(precision=0.26665167245767174, recall=0.2299859123345547, fmeasure=0.22831482511807025), high=Score(precision=0.32164933544731733, recall=0.25466261649215866, fmeasure=0.25232966126886747)),
 'rougeLsum': AggregateS